<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/suno-ai/bark.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 16.0 MB/s eta 0:00:00


In [3]:
import requests
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from bs4 import BeautifulSoup
import re

In [4]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [5]:
from datetime import datetime
def search_urls_space_news():
  day = int(datetime.now().day)
  month= int(datetime.now().month)
  year = int(datetime.now().year)

  URL = f"https://www.google.com/search?q=space.com+latest+news&tbm=nws"
  print(URL)
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [6]:
hrefs = search_urls_space_news()

https://www.google.com/search?q=space.com+latest+news&tbm=nws


In [7]:
print(hrefs)

['/?sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQOwgC', '/search?q=space.com+latest+news&tbm=nws&sca_esv=569384727&ie=UTF-8&gbv=1&sei=hHwWZbjjOuzAkPIPmYu0wAY', '/search?q=space.com+latest+news&sca_esv=569384727&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQ_AUIBSgA', '/search?q=space.com+latest+news&sca_esv=569384727&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQ_AUIBygC', '/search?q=space.com+latest+news&sca_esv=569384727&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQ_AUICCgD', 'https://maps.google.com/maps?q=space.com+latest+news&um=1&ie=UTF-8&sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQ_AUICSgE', '/search?q=space.com+latest+news&sca_esv=569384727&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQ_AUICigF', '/search?q=space.com+latest+news&sca_esv=569384727&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwi4tY_spc-BAxVsIEQIHZkFDWgQ_AUICygG', '/advanced_search', '/search?q=space.com+latest+news&sca_esv=56

In [8]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [9]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [10]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.space.com/historic-donut-black-hole-is-spinning-new-study-finds',
 'https://www.space.com/spacex-starlink-launch-group-7-3',
 'https://www.space.com/harvest-full-moon-will-illuminate-the-sky-on-september-29',
 'https://www.space.com/osiris-rex-asteroid-sample-caniser-lid-lifted-photos',
 'https://www.space.com/china-yaogan-spy-satellite-launch-september-2023',
 'https://www.space.com/spacex-launch-psyche-asteroid-mission-one-week',
 'https://www.space.com/iran-launches-noor-3-imaging-satellite-september-2023',
 'https://www.google.com/search?q%3Dspace.com%2Blatest%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://www.space.com/gravity-affects-matter-antimatter-similarly',
 'https://news.yahoo.com/last-supermoon-2023-know-week-223424273.html',
 'https://www.space.com/james-webb-space-telescope-evolved-galaxy-early-universe']

In [11]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [12]:
titles, articles = get_text_from_urls(cleaned_urls)

In [13]:
articles[0]

'When you purchase through links on our site, we may earn an affiliate commission. Here’s how it works. The historic \'donut\' black hole M87* is, in fact, spinning. The supermassive black hole M87*, which rose to fame in 2019 when it became the first void to be imaged and revealed a fuzzy orange donut (then later sharpened by AI into a skinny ring), is now confirmed to be spinning. The announcement of this update came on Wednesday (Sept. 27). However, as to how fast M87* is spinning? That\'s not yet known. For two decades, a network of radio telescopes have been eyeing the black hole, which resides in the heart of the Messier 87 (M87) galaxy about 55 million light-years away from Earth in the constellation Virgo. These instruments have been especially intrigued by a powerful jet of radiation and particles blasting from the black hole\'s poles, and according to new results, that relativistic jet appears to be swinging like a pendulum on a 11-year cycle. Scientists say this is because o

Split text from the article into chunks

In [14]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['When you purchase through links on our site, we may earn an affiliate commission.',
 ' Here’s how it works.',
 ' "Based on our results astronomers must rethink our understanding of the formation of the first galaxies and how galaxy evolution occurred over the past 10 billion years.',
 '" What did galaxies in the early universe look like?',
 ' Surprisingly close to our own Milky Way, according to the latest findings from the James Webb Space Telescope (JWST), whose unprecedented infrared eye has been rewriting what we thought we knew about the early universe.',
 '\xa0 Astronomers have long thought that newly minted galaxies that began merging together \xa0just after the Big Bang, about 13.',
 '7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings.',
 ' Those galactic features were thought to form during a time at least six billion years after the Big Bang.',
 " According to the new study, however, these delicate shapes could've manife

In [15]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [16]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [17]:
joined_chunks[0][1]

" Specifically, the effect comes to light with the team's new study because the spin axis of a black hole is not perfectly aligned with the rotation axis of the surrounding accretion disk from which the black hole sucks stellar material.  This triggers the black hole's jets to wobble ever so slightly, which was what was measured in the new study.  —\xa0Milky Way vs M87: Event Horizon Telescope photos show 2 very different monster black holes —\xa0Astronomers may have discovered the closest black holes to Earth —\xa0Black holes keep 'burping up' stars they destroyed years earlier, and astronomers don't know why The specific processes that cause black holes to spin are not very well understood.  A leading theory suggests smaller black holes form by feeding on star matter through an accretion disk, which causes them to spin rapidly.  Over eons, they are thought to collide and eventually merge to form supermassive black holes. \xa0 These second generation black holes are expected to spin s

Summarize all chunks

In [18]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    print(len(article_chunks))
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [19]:
summarized_texts = summarize_chunks(joined_chunks)

2
2
3
2
2
2
2
3
1
2


In [20]:
summarized_texts[4]

" China has launched another new Yaogan reconnaissance satellite . A Long March 4C rocket lifted off from Jiuquan Satellite Launch Center in the Gobi Desert . The launch was China's 45th of 2023 . It follows the launch of the third Yaogan satellite earlier this month .  September 28, 2023 is 20 years since the birth of the Tiangong space station . This month is the 20th anniversary of the first manned mission to the moon ."

In [21]:
all_summarized_text = ""
for summarized_text in summarized_texts:
  all_summarized_text += summarized_text

In [22]:
all_summarized_text

' Supermassive black hole M87* is now confirmed to be spinning, according to a new study . A powerful jet of radiation and particles from the black hole\'s poles appears to be swinging like a pendulum on a 11-year cycle . Scientists say this is because of gravitational interactions between the spinning black hole, which is 6.5 billion times more massive than the sun, and the disk of material around it .  The specific processes that cause black holes to spin are not very well understood . A leading theory suggests smaller black holes form by feeding on star matter through an accretion disk, which causes them to spin rapidly . Over eons, they are thought to collide and eventually merge to form supermassive black holes . A SpaceX Falcon 9 rocket launched from California early Monday morning (Sept. 25) carrying 21 Starlink satellites to orbit . The launch was streamed live via SpaceX\'s account on X (formerly Twitter) The Falcon 9\'s first stage returned to Earth safely, landing on a Space

Create a news article with all of the summarized texts

In [23]:
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [ ]:
# summarized_texts

[' Astronomers have long thought that newly minted galaxies that began merging together just after the Big Bang, about 13. 7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings . The new findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data .  James Webb Space Telescope could detect life on Earth from across the galaxy, new study suggests . The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years . The research is described in a paper published Sept. 22 in The Astrophysical Journal .',
 " A SpaceX Falcon 9 rocket launched from California early Monday morning (Sept. 25) carrying 21 Starlink satellites to orbit . The launch was streamed live via SpaceX's account on X (formerly Twitter) The Falcon 9's first stage returned to Earth safely, landing on a SpaceX drone ship at sea about 8. 5 minutes after launch . 

In [ ]:
# def make_news_summary(all_chunks):
#   summary_space_news = ""
#   for chunk in all_chunks:
#     inputs = tokenizer(f"Make this text like it will be used in space news report: {chunk}", return_tensors='pt', truncation=True, padding=True)
#     outputs = model.generate(**inputs, max_length=512, min_length=len(inputs['input_ids'][0]), temperature=0.25)

#     # Convert the generated output tensor to a string
#     summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=512)
#     print(outputs)
#     summary_space_news += summary_text + '\n'
#   return summary_space_news

In [ ]:
# summary_space_news = make_news_summary(summarized_texts)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


tensor([[    0, 15001,  3114,   277,    43,   307,   816,    24,  6164,     3,
          8215,    26,  7466,  8606,    15,     7,    24,  1553, 12147,    53,
           544,   131,   227,     8,  2734, 19330,     6,    81,  8808,   489,
          2108,   203,   977,     6,   130,   396, 18659,    12,  8125,   136,
         20815,  5278,   114, 17723,  6026,     6,  6448,    42, 10631,     5,
            37,   126,  7469,   130,     3,   390,    30,    46,  1693,    13,
          6180,  3301,   948,  7466,  8606,    15,     7,     6,    84,     3,
         12466,   277,   497,    19,     8,  2630,  3106,    24,    65,   118,
          7463,    28,   446,   518,  4209,   331,     5,    37,  7469,  3130,
             3,     9,   174,    21,   126,   912,    24,  3209,   149,  7466,
          8606,    15,     7, 14021,   147,     8,   657,   335,  2108,   203,
             5,    37,  7469,   130,     3,   390,    30,    46,  1693,    13,
          6180,  3301,   948,  7466,  8606,    15,  

In [ ]:
#summary_space_news

'Astronomers have long thought that newly minted galaxies that began merging together just after the Big Bang, about 13. 7 billion years ago, were too fragile to boast any noticeable structures like spiral arms, bars or rings. The new findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data. The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years. The findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data. The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years. The findings were based on an analysis of 3,956 galaxies, which astronomers say is the biggest sample that has been studied with JWST data. The findings suggest a need for new ideas that explain how galaxies evolved over the past 10 billion years.\nSpaceX launches rocket c

NLP for news reading

In [24]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio

In [25]:
preload_models()

Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:01<00:00, 71.3MB/s]


Text to chunks and then to tts
Then append audio array

In [28]:
full_audio_array = []
for i, summarized_text in enumerate(summarized_texts):
  audio_array = generate_audio(summarized_text, history_prompt="v2/en_speaker_1")
  if i==0:
    full_audio_array.append(audio_array)
  else:
    full_audio_array.extend(audio_array)
Audio(full_audio_array, rate=SAMPLE_RATE)

100%|██████████| 36/36 [00:35<00:00,  1.02it/s]


ValueError: ignored

In [27]:
print(full_audio_array)

[array([ 5.0278555e-04,  1.4294132e-04, -1.7490180e-04, ...,
        4.0170853e-06,  1.6545324e-05,  1.9026629e-05], dtype=float32), array([0.00107054, 0.00060457, 0.00022582, ..., 0.00433661, 0.00411826,
       0.00437741], dtype=float32), array([7.9013663e-04, 3.9282689e-04, 3.1310439e-05, ..., 5.9088543e-03,
       3.8350120e-03, 2.4492440e-03], dtype=float32), array([ 7.3120650e-04,  3.2239855e-04, -1.3466633e-05, ...,
        4.8473589e-03,  4.7381828e-03,  4.5307036e-03], dtype=float32), array([ 7.9405319e-04,  3.8444475e-04,  4.9832699e-05, ...,
        1.1542306e-02,  1.6716292e-02, -4.1048639e-02], dtype=float32), array([ 7.1834237e-04,  3.1795740e-04, -2.1537970e-05, ...,
        3.6691006e-02, -2.1605922e-02, -2.3950431e-03], dtype=float32), array([8.9894817e-04, 4.5947460e-04, 7.1073897e-05, ..., 2.0519078e-03,
       3.8910874e-03, 3.4106532e-05], dtype=float32), array([ 0.00204631,  0.00124314,  0.00109829, ..., -0.00043059,
       -0.00062042, -0.00083867], dtype=float32

In [31]:
import scipy.io.wavfile as wavf
import numpy as np
full_audio_array_np = np.array(full_audio_array)
scaled = np.int16(full_audio_array_np/np.max(np.abs(full_audio_array_np)) * 32767)
fs = 44100
out_f = 'out.wav'

wavf.write(out_f, fs, scaled)

<ipython-input-31-55b64e2119cb>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  full_audio_array_np = np.array(full_audio_array)


ValueError: ignored